In [ ]:
from time import sleep
import google.generativeai as genai
import configparser
import json
import re
import pandas as pd

In [2]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
consulta = """
"Buenas tardes, soy Extranjero Venezolano y antes no tenía DNI y me otorgaron el CUIL/CUIT anses que tienen actualmente, pero, ya me llego el DNI y actualice mi CUIL/CUIT
Datos actuales:
  DNI: 96372499
  CUIL/CUIT: 27963724999

COMO PODRÁ DARSE CUENTA NECESITO QUE MI CUENTA SE ACTUALICE CON MIS DOCUMENTOS ACTUALIZADOS 
"""

prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
    )

print(response.text)

Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [ ]:
data = pd.read_excel('encontraste_lo_que_buscabas-procesado.xlsx')
data.head(5)

In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    # ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)    
    consulta = data['Mensaje'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 
    print(consulta)   
    prompt = f"""
    Extraer un resumen de lo que quiere el remitente en la siguiente texto, que está delimitada por tres comillas invertidas.
    Extraer los temas principales de la consulta.
    Haga que cada elemento tenga una o dos palabras. Formatee su respuesta como una lista de elementos separados por comas.
    Indique el sentimiento general del texto.
    Indique si el remitente está expresando enojo.    

    Texto: '''{consulta}'''
    Salida en formato JSON:
    "Resumen":
    "Temas": 
    "Sentimientos": 
    "Enojo": "Si/No"
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Resumen'] = analisis.get('Resumen', 'None')
            data.loc[i, 'Temas'] = analisis.get('Temas', [])            
            data.loc[i, 'Sentimientos'] = analisis.get('Sentimientos', 'None')
            data.loc[i, 'Enojo'] = analisis.get('Enojo', 'None')            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")

    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('encontraste_lo_que_buscabas-procesado.xlsx', index=False)


In [ ]:
data = pd.read_excel('encontraste_lo_que_buscabas-procesado.xlsx')
data.head(5)

In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    consulta = data['Mensaje'][i]
    url_origen = data['URL'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 


    # Expresión regular para capturar desde "/inicio/" hasta antes de los parámetros
    pattern = r"/inicio/[^?]+"

    # Buscar el patrón en la URL
    match = re.search(pattern, url_origen)

    # Obtener el resultado
    if match:
        url_result = match.group(0)
    else:
        url_result = url_origen

    print(i, url_result)
    print(consulta)
    prompt = f"""
    Se le proporcionarán consultas de vecinos al sitio web de la municipalidad.
    Clasifique cada consulta en una categoría principal y una categoría secundaria en base al texto y a la URL de origen, que está delimitado por tres comillas invertidas.
    Categorías principales: Alumbrado, Licencia de conducir, Multas, Movilidad, Salud, Consulta general.
    
    Categorías secundarias de Alumbrado:
    Reclamo
    Robo
    Indefinido

    Categorías secundarias de Licencia de conducir:
    Sin turnos
    Error en el sistema
    Indefinido

    Categorías secundarias de Multas:
    Sin turnos
    Error en el sistema
    Descargo
    Titularidad
    Indefinido
    
    Categorías secundarias de Movilidad:
    Medio Boleto
    Sube
    Sin turnos
    Bicicleta
    Indefinido

    Categorías secundarias de Salud:
    Sin turnos
    Error en el sistema
    Reclamo
    Indefinido

    Categorías secundarias de Consulta general:
    Contraseña
    Actualización de DNI/CUIL
    Pedido de información
    Reclamo
    Error en el sistema
    Sin turnos
    Indefinido
    
    Texto: '''{consulta}'''
    URL origen: '''{url_result}'''
    
    Salida en formato JSON:
    "Principal":
    "Secundaria": 
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Principal'] = analisis.get('Principal', 'None')
            data.loc[i, 'Secundaria'] = analisis.get('Secundaria', [])            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")
        
    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
            
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('encontraste_lo_que_buscabas-procesado.xlsx', index=False)
